In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import faiss
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import re
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.utils import resample
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
from scipy.sparse import hstack, csr_matrix
from scipy.stats import uniform, randint


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [2]:
data = pd.read_csv('Datasets/features.csv')

In [3]:
data.columns

Index(['Label', 'Preprocessed_Content', 'sentiment', 'hate_speech_count',
       'pos_tags', 'word2vec', 'sbert_embedding', 'lexical_diversity',
       'sentence_complexity'],
      dtype='object')

In [4]:
len(data)

407364

In [5]:
data['Label'].value_counts()

Label
0    333185
1     74179
Name: count, dtype: int64

In [6]:
from sklearn.utils import resample

class_0 = data[data['Label'] == 0]
class_1 = data[data['Label'] == 1]

class_0_sampled = resample(class_0, n_samples=25000, random_state=42)
class_1_sampled = resample(class_1, n_samples=25000, random_state=42)

balanced_data = pd.concat([class_0_sampled, class_1_sampled])

balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

print(balanced_data['Label'].value_counts())


Label
1    25000
0    25000
Name: count, dtype: int64


In [7]:
train_data = balanced_data

In [8]:
def convert_embedding(embedding):
    if isinstance(embedding, np.ndarray):
        return embedding
    elif isinstance(embedding, list):
        return np.array(embedding, dtype=np.float32)
    elif isinstance(embedding, str):
        embedding = embedding.strip("[]")
        embedding = np.array([float(x) for x in embedding.split()], dtype=np.float32)
        return embedding
    else:
        return np.zeros(768, dtype=np.float32)

train_data['sbert_embedding'] = train_data['sbert_embedding'].apply(convert_embedding)

embeddings = np.stack(train_data['sbert_embedding'].values)

print("Embeddings Shape:", embeddings.shape)


Embeddings Shape: (50000, 384)


In [9]:
faiss_index = faiss.IndexFlatL2(embeddings.shape[1])
faiss_index.add(embeddings)

In [10]:
tfidf = TfidfVectorizer(ngram_range=(2, 3), stop_words='english')
X_tfidf = tfidf.fit_transform(train_data['Preprocessed_Content'])

In [11]:
vectorizer = CountVectorizer(ngram_range=(2, 3), stop_words='english')
X_bow = vectorizer.fit_transform(train_data['Preprocessed_Content'])

In [12]:
X = hstack([
    X_tfidf,
    X_bow,  
    embeddings,
    train_data[['sentiment', 'hate_speech_count', 'lexical_diversity', 'sentence_complexity']].values
])

Y = train_data['Label']

In [13]:
scaler = StandardScaler()
numerical_features = train_data[['sentiment', 'hate_speech_count', 'lexical_diversity', 'sentence_complexity']].values
numerical_features_scaled = scaler.fit_transform(numerical_features)

scaled_sparse = csr_matrix(numerical_features_scaled)

In [14]:
smote = SMOTE(random_state=42)
X_res, Y_res = smote.fit_resample(X, Y)

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X_res, Y_res, test_size=0.2, random_state=42)

In [16]:
class_weights = compute_class_weight('balanced', classes=np.unique(Y_res), y=Y_res)
class_weight_dict = dict(zip(np.unique(Y_res), class_weights))

In [17]:
log_reg_params = {
    'C': np.logspace(-4, 4, 10),
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}

log_reg_model = RandomizedSearchCV(LogisticRegression(max_iter=1000), log_reg_params, cv=3, scoring='f1', n_iter=10, n_jobs=-1)
log_reg_model.fit(X_train, Y_train)

print("Best Logistic Regression Params:", log_reg_model.best_params_)

Best Logistic Regression Params: {'solver': 'liblinear', 'penalty': 'l2', 'C': 2.782559402207126}


In [18]:
log_reg_pred = log_reg_model.best_estimator_.predict(X_test)
print("Logistic Regression Performance:")
print(classification_report(Y_test, log_reg_pred))

Logistic Regression Performance:
              precision    recall  f1-score   support

           0       0.83      0.80      0.82      5028
           1       0.81      0.84      0.82      4972

    accuracy                           0.82     10000
   macro avg       0.82      0.82      0.82     10000
weighted avg       0.82      0.82      0.82     10000



In [19]:
svm_params = {
    'C': np.logspace(-2, 2, 5),
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

svm_model = RandomizedSearchCV(SVC(), svm_params, cv=3, scoring='f1', n_iter=10, n_jobs=-1)
svm_model.fit(X_train, Y_train)

print("Best SVM Params:", svm_model.best_params_)
svm_pred = svm_model.best_estimator_.predict(X_test)
print("SVM Performance:")
print(classification_report(Y_test, svm_pred))

KeyboardInterrupt: 

In [ ]:
rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True]
}

rf_model = RandomizedSearchCV(RandomForestClassifier(random_state=42), rf_params, cv=3, scoring='f1', n_iter=10, n_jobs=-1)
rf_model.fit(X_train, Y_train)

print("Best Random Forest Params:", rf_model.best_params_)
rf_pred = rf_model.best_estimator_.predict(X_test)
print("Random Forest Performance:")
print(classification_report(Y_test, rf_pred))

Fitting 2 folds for each of 3 candidates, totalling 6 fits
Random Forest Performance:
              precision    recall  f1-score   support

           0       0.84      0.89      0.86      3043
           1       0.88      0.83      0.85      2957

    accuracy                           0.86      6000
   macro avg       0.86      0.86      0.86      6000
weighted avg       0.86      0.86      0.86      6000

